# Compute Field Parameters

SageMath notebook to compute the field parameters to implement traits `ark_ff::fields::FpParameters` and `ark_ff::fields::FftParameters`

Run using SageMath 9.0 (warning: will fail in python)

In [1]:
def to_bytes_hex_string(parameter, block_size = 16):
    hex_representation = str(hex(parameter)).split('x')[-1]
    padded = hex_representation.rjust(64, '0')
    output = '0x{}, 0x{}, 0x{}, 0x{},'.format(padded[48:64], padded[32:48], padded[16:32], padded[0:16])
    return output

In [2]:
def square_mult_reduce(base, power, modulo):
    power_string = str(bin(power)).split('b')[-1]
    number_of_rounds = len(power_string)
    result = base
    for i in range(1, number_of_rounds):
        result = result ** 2 % modulo
        if power_string[i] == '1':
            result = result * base % modulo
            
    return result

In [6]:
def compute_t_s(modulus):
    T = modulus - 1
    S = 0
    while T%2==0:
        S = S+1
        T = T/2 
    return T, S

In [13]:
def compute_fp_parameters(modulus, generator, closest_two_power_64=256):
    parameters = {'MODULUS': modulus}
    parameters['MODULUS_BITS'] = len(str(bin(modulus)).split('b')[-1])
    R = 2**closest_two_power_64 % modulus
    parameters['R'] = R
    parameters['R2'] = (2**closest_two_power_64)**2 % modulus
    parameters['MODULUS_MINUS_ONE_DIV_TWO'] = (modulus - 1)/2
    T, S = compute_t_s(modulus)
    parameters['T'] = T
    parameters['T_MINUS_ONE_DIV_TWO'] = (T - 1)/2
    parameters['GENERATOR'] = generator*R % modulus
    parameters['INV'] = -modulus**(-1) % 2**64
    parameters['TWO_ADICITY'] = S
    parameters['TWO_ADIC_ROOT_OF_UNITY'] = square_mult_reduce(generator, T, modulus)*R % modulus
    
    return parameters

In [14]:
base_field_parameters = compute_fp_parameters(2**251 + 17 * 2**192 + 1, 3); base_field_parameters

{'MODULUS': 3618502788666131213697322783095070105623107215331596699973092056135872020481,
 'MODULUS_BITS': 252,
 'R': 3618502788666127798953978732740734578953660990361066340291730267701097005025,
 'R2': 3618203731877326472068475314175255648288435919629678774199199170643399541761,
 'MODULUS_MINUS_ONE_DIV_TWO': 1809251394333065606848661391547535052811553607665798349986546028067936010240,
 'T': 576460752303423505,
 'T_MINUS_ONE_DIV_TWO': 288230376151711752,
 'GENERATOR': 3618502788666120969467290632032063525614768540420005620929006690831546974113,
 'INV': 18446744073709551615,
 'TWO_ADICITY': 192,
 'TWO_ADIC_ROOT_OF_UNITY': 3226581589306417752488187218501353947335076990068651027982507683041266286040}

In [17]:
EC_GROUP_ORDER = 3618502788666131213697322783095070105526743751716087489154079457884512865583 #EC group order, see StarkNet doc
scalar_field_parameters = compute_fp_parameters(EC_GROUP_ORDER, 3); scalar_field_parameters

{'MODULUS': 3618502788666131213697322783095070105526743751716087489154079457884512865583,
 'MODULUS_BITS': 252,
 'R': 3618502788666127798953978732740734581940928362441851875681120813493230806863,
 'R2': 3551179599793676589179020395559770927735842444383898408469482222901708744845,
 'MODULUS_MINUS_ONE_DIV_TWO': 1809251394333065606848661391547535052763371875858043744577039728942256432791,
 'T': 1809251394333065606848661391547535052763371875858043744577039728942256432791,
 'T_MINUS_ONE_DIV_TWO': 904625697166532803424330695773767526381685937929021872288519864471128216395,
 'GENERATOR': 3618502788666120969467290632032063534769297583893380648735203524710666689423,
 'INV': 13504954208620504625,
 'TWO_ADICITY': 1,
 'TWO_ADIC_ROOT_OF_UNITY': 3414743344050354335523585815389274235613472958644391282058720}